# Crossmatch SDSS to all

Taking the 9258  S82 QSO from SDSS (DBQSO). Seek to find the PS1, PTF, CRTS, data. Mostly based on AA_crossmatch_to_all,  but doing the outer join, to have as much data as possible, especially that in the end we only use SDSS-PS1,... So relax the hitherto stringent condition imposed by inner join that the quasar has to have SDSS-PS1-PTF-CRTS data altogether ....  <a id='toc'></a>
    
   * cross-match, regardless of filter:
    * [SDSS-PS1](#sdss_ps1)
    * [SDSS-PTF](#sdss_ptf)
    * [SDSS-CRTS](#sdss_crts)
    * [SDSS-CRTS-PTF-PS1 : outer join](#join)
   * store per object [time, mag, magerr, filter, survey] in dp2/real_outer_join/
   * next step : combine all to SDSS-r master band 

In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import os
import pandas as pd 
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.table import  hstack
from astropy.table import  vstack
from astropy.table import Column
from astropy.table import join

## SDSS-PS1

In [2]:
ps1_qso_schneider = os.listdir('../raw_data/schneider/')

# grab ra, dec from filenames 
ra_ps1 = np.array([coord[:9] for coord in ps1_qso_schneider])
dec_ps1 = np.array([coord[9:] for coord in ps1_qso_schneider])

# make into AstroPy - digestible format.. 
ra_ps1_split = [ra[:2] + ' '+ ra[2:4] + ' '+ra[4:] for ra in ra_ps1]
dec_ps1_split = [dec[:3]+ ' '+ dec[3:5]+ ' ' + dec[5:] for dec in dec_ps1]

# combining the two.. 
radec_ps1_split = np.zeros_like(ra_ps1_split, dtype='U32')
for i in range(len(ra_ps1_split)):
    radec_ps1_split[i] = ra_ps1_split[i] + ' ' + dec_ps1_split[i]

c = SkyCoord(radec_ps1_split, frame='icrs', unit=(u.hourangle, u.deg)) 

# Check how many SDSS S82 DR9 QSOs have PS1 matches 
File = '../../CRTS_PROJECT/catalogs_SDSS/DB_QSO_S82.dat'
colnames = ['dbID', 'ra', 'dec', 'SDR5ID', 'M_i', 'M_i_corr', 'redshift',
                   'mass_BH', 'Lbol', 'u', 'g', 'r', 'i', 'z', 'Au']

# 9258 QSO ....
SDSS_DB_QSO= Table.read(File, format='ascii', names = colnames)
sdss_catalog = SkyCoord(ra=SDSS_DB_QSO['ra']*u.degree, dec=SDSS_DB_QSO['dec']*u.degree)


# Match Schneider PS1 to SDSS S82 DR7 : 
# better : match SDSS to PS1,  
# because there is less 
idx, sep2d, dist3d = sdss_catalog.match_to_catalog_sky(c) 
SDSS_DB_QSO['d2d_arcsec']=sep2d.value*3600

ps1id = np.asarray(ps1_qso_schneider)[idx]
ps1ra = c[idx].ra.deg
ps1dec = c[idx].dec.deg
ps1 = Table([ps1id, ps1ra, ps1dec], names=['ps1id', 'ps1ra', 'ps1dec'])

sdss_ps1 = hstack([SDSS_DB_QSO, ps1])


In [3]:
matchRadiusArcsec = 0.5
fname = 'SDSS_DB9_PS1_QSO_database.txt'
mask = sdss_ps1['d2d_arcsec'] < matchRadiusArcsec
print('Of ', len(SDSS_DB_QSO) , ' SDSS DB9 QSO, ', 
      np.sum(mask), 
    ' has a PS1 match within ', matchRadiusArcsec, ' arcseconds ')

print('We save these to ', fname)
sdss_ps1[mask].write(fname, format='ascii', overwrite=True)

Of  9258  SDSS DB9 QSO,  9248  has a PS1 match within  0.5  arcseconds 
We save these to  SDSS_DB9_PS1_QSO_database.txt


## SDSS-PTF

In [5]:
ptf_lcs =  Table.read('../../CRTS_PROJECT/IPAC_output_tables/\
Ipac_PTF_lightcurves_QSO_S82.tbl', 
                     format='ipac')



In [6]:
# I use the 'ra' key that corresponds to  the SDSS ra, since ra_01 : 
# the PTF ra , may differ from chip to chip, just as oid . 
# For these quasars there seems to be only one edge-case 
# ( number of unique ra is 6471, and oid : 6472, ra_01: 6480)

from astropy import units as u
from astropy.coordinates import SkyCoord

ptf_ra = ptf_lcs['ra'].data.data
ptf_dec = ptf_lcs['dec'].data.data
print(len(np.unique(ptf_ra)))

# Match SDSS to PTF, since there is less PTF than SDSS 

# SDSS 9258 DR9 QSO 
File = '../../CRTS_PROJECT/catalogs_SDSS/DB_QSO_S82.dat'
colnames = ['dbID', 'ra', 'dec', 'SDR5ID', 'M_i', 'M_i_corr', 'redshift',
                   'mass_BH', 'Lbol', 'u', 'g', 'r', 'i', 'z', 'Au']
SDSS_DB_QSO= Table.read(File, format='ascii', names = colnames)
sdss_catalog = SkyCoord(ra=SDSS_DB_QSO['ra']*u.degree, 
                        dec=SDSS_DB_QSO['dec']*u.degree)
ptf_catalog = SkyCoord(ptf_ra*u.degree,ptf_dec*u.degree, 
                       frame='icrs') 
idx, sep2d, dist3d = sdss_catalog.match_to_catalog_sky(ptf_catalog) 
SDSS_DB_QSO['d2d_arcsec']=sep2d.value*3600


# take the PTF table by matching indices 
ptf =  Table([ptf_ra[idx], ptf_dec[idx], ptf_lcs['oid'].data.data[idx]],
            names = ['ra_sdss', 'dec_sdss', 'ptf_oid'])

# combine 
sdss_ptf = hstack([SDSS_DB_QSO, ptf])

# only save those that are well matchewd ... 
matchRadiusArcsec = 0.5
fname = 'SDSS_DB9_PTF_QSO_database.txt'
mask = sdss_ptf['d2d_arcsec'] < matchRadiusArcsec
print('Of ', len(SDSS_DB_QSO) , ' SDSS DB9 QSO, ', 
      np.sum(mask), 
    ' has a PTF match within ', matchRadiusArcsec, ' arcseconds ')

print('We save these to ', fname)
sdss_ptf[mask].write(fname, format='ascii', overwrite=True)

6471
Of  9258  SDSS DB9 QSO,  6455  has a PTF match within  0.5  arcseconds 
We save these to  SDSS_DB9_PTF_QSO_database.txt


## SDSS-CRTS

In [7]:
dir_in = '../../CRTS_PROJECT/raw_LC_CRTS/QSO_CRTS/'
QSO_arr = np.array(os.listdir(dir_in))
print('We have %d CRTS qso files to start with ' % len(QSO_arr))

# ignore the two files that are not lightcurves
a = np.array([len(entry) for entry in QSO_arr])
QSO_list = list(QSO_arr)
QSO_list.remove('message.txt')
QSO_list.remove('file.list')

c = 0 
empty_files = []
for obj in QSO_list:
    address = dir_in + obj
    f = open(address, 'r')
    if f.readline() == '' :
        c += 1
        empty_files.append(obj)
        
print('We have %d empty files' % c)    

crts_qso = np.array(QSO_list)[~np.in1d(QSO_list, empty_files)]  
print('We are left with ', len(crts_qso), 'non-empty files ... ')

# grab ra, dec from filenames 
ra_crts = np.array([coord[:9] for coord in crts_qso])
dec_crts = np.array([coord[9:-4] for coord in crts_qso])

# make into AstroPy - digestible format.. 
ra_crts_split = [ra[:2] + ' '+ ra[2:4] + ' '+ra[4:] for ra in ra_crts]
dec_crts_split = [dec[:3]+ ' '+ dec[3:5]+ ' ' + dec[5:] for dec in dec_crts]

# combining the two.. 
radec_crts_split = np.zeros_like(ra_crts_split, dtype='U32')
for i in range(len(ra_crts_split)):
    radec_crts_split[i] = ra_crts_split[i] + ' ' + dec_crts_split[i]
    
crts_catalog = SkyCoord(radec_crts_split, frame='icrs', unit=(u.hourangle, u.deg))

# Check how many  of 9258  SDSS S82 DR9 QSOs have CRTS matches 
File = '../../CRTS_PROJECT/catalogs_SDSS/DB_QSO_S82.dat'
colnames = ['dbID', 'ra', 'dec', 'SDR5ID', 'M_i', 'M_i_corr', 'redshift',
                   'mass_BH', 'Lbol', 'u', 'g', 'r', 'i', 'z', 'Au']
SDSS_DB_QSO= Table.read(File, format='ascii', names = colnames)
sdss_catalog = SkyCoord(ra=SDSS_DB_QSO['ra']*u.degree, 
                        dec=SDSS_DB_QSO['dec']*u.degree)

# better : match SDSS to CRTS,  
# because there is less 
idx, sep2d, dist3d = sdss_catalog.match_to_catalog_sky(crts_catalog) 
SDSS_DB_QSO['d2d_arcsec']=sep2d.value*3600

crtsRA = crts_catalog[idx].ra.deg
crtsDEC = crts_catalog[idx].dec.deg
crtsID = crts_qso[idx]
crts = Table([crtsID, crtsRA, crtsDEC], names=['crtsID', 'crtsRA', 'crtsDEC'])

sdss_crts = hstack([SDSS_DB_QSO, crts])

matchRadiusArcsec = 0.5
fname = 'SDSS_DB9_CRTS_QSO_database.txt'
mask = sdss_crts['d2d_arcsec'] < matchRadiusArcsec
print('Of ', len(SDSS_DB_QSO) , ' SDSS DB9 QSO, ', 
      np.sum(mask), 
    ' has a CRTS match within ', matchRadiusArcsec, ' arcseconds ')

print('We save these to ', fname)
sdss_crts[mask].write(fname, format='ascii', overwrite=True)

We have 7933 CRTS qso files to start with 
We have 179 empty files
We are left with  7752 non-empty files ... 
Of  9258  SDSS DB9 QSO,  7737  has a CRTS match within  0.5  arcseconds 
We save these to  SDSS_DB9_CRTS_QSO_database.txt


## SDSS-PS1-PTF-CRTS :  outer join

In [9]:
sdss_ps1 = Table.read('SDSS_DB9_PS1_QSO_database.txt', format='ascii')
sdss_ptf = Table.read('SDSS_DB9_PTF_QSO_database.txt', format='ascii')
sdss_crts = Table.read('SDSS_DB9_CRTS_QSO_database.txt', format='ascii')
print(len(sdss_ps1))
print(len(sdss_ptf))
print(len(sdss_crts))

9248
6455
7737


Loop over all SDSS : the DB_QSO  containing 9258  Quasars . If there is any match for SDSS-PS1, add PS1 data. If any match in SDSS-PTF, add PTF data, etc. 

In [11]:
sdss = SDSS_DB_QSO

In [ ]:
# SDSS colnames 
colnames = []
for f in 'ugriz':
    colnames.append(f+'mjd')
    colnames.append(f+ 'mag')
    colnames.append(f+'err')
colnames.append('RA')
colnames.append('DEC')

In [42]:
lc

mag,magerr,mjd,filter,survey
float64,float64,float64,str1,str4
20.693,0.08,51075.305869,u,sdss
20.699,0.076,52196.306378,u,sdss
20.822,0.078,52197.319485,u,sdss
20.734,0.081,52225.169672,u,sdss
20.74,0.074,52231.205454,u,sdss
20.854,0.156,52253.191626,u,sdss
20.892,0.104,52557.313456,u,sdss
20.957,0.097,52578.209542,u,sdss
20.805,0.092,52908.288024,u,sdss


In [53]:
sdss_dir = '../../CRTS_PROJECT/raw_LC_SDSS/QSO_S82/'
ps1_dir =   '../raw_data/schneider/'
crts_dir = '../../CRTS_PROJECT/raw_LC_CRTS/QSO_CRTS/'
out_dir =  '../dp2/real_outer_join/'

i = 0 
for sdss_name in sdss['dbID']: 
    print(i, 'sdss dbID: ', sdss_name)
    
    # initialize an object to hold all photometry data 
    lc = Table() 

    # start from SDSS data 
    sdss_lc = Table.read(sdss_dir+str(sdss_name.astype(int)), 
                         format='ascii',names=colnames)
    # recast the sdss_lc from umag,umjd,uerr,  gmag,gmjd,gerr   
    # into   mag,mjd,err,filter.... \
    
    for f in 'ugriz':
        single = Table(sdss_lc[[f+'mag', f+'err', f+'mjd']], 
                       names = ['mag', 'magerr', 'mjd'])
        # don't store epochs  with no measurement in a given filter .... 
        single = single[single['mag'].data > 0] 
        single['filter'] = f
        single['survey'] = 'sdss'
        lc = vstack([lc,single])
        
  # add PTF info  
    if np.in1d(sdss_name, sdss_ptf['dbID'].data)[0] : 
        row = np.where(sdss_ptf['dbID'].data == sdss_name)[0][0]
        ptf_name = sdss_ptf[row]['ptf_oid'].astype(int)
        print('  ptf :  ', ptf_name)
        ptf_lc = ptf_lcs[ptf_lcs['oid'].data.data == ptf_name]

        # PTF uses g, and R : g-band during dark time, and R-band during bright time
        # (1=g-band, 2=R-band). Majority is g-band (12 000 epochs), the rest is 
        # R-band (147 000 epochs). 

        ptf_lc_ = Table(ptf_lc['obsmjd', 'mag_autocorr', 'magerr_auto','fid'],
                        names=['mjd', 'mag', 'magerr','filter'])
        ptf_lc_['filter'] = ptf_lc_['filter'].astype(str)
        ptf_lc_['survey'] = 'ptf'

        # stack that
        lc = vstack([lc,ptf_lc_])
        
    # add CRTS info  : white light 
    if np.in1d(sdss_name, sdss_crts['dbID'].data)[0] : 
        row = np.where(sdss_crts['dbID'].data == sdss_name)[0][0]
        crts_name = sdss_crts[row]['crtsID']
        print('  crts :  ',crts_name)
        crts_lc = Table.read(crts_dir+crts_name, format='ascii', 
                            names=['mjd', 'mag', 'magerr', '0'])
        crts_lc_ = crts_lc[['mjd', 'mag', 'magerr']]
        crts_lc_['filter'] = 'w'
        crts_lc_['survey'] = 'crts'

        # stack that
        lc = vstack([lc,crts_lc_])
        
        
    # add PS1 : bit more complicated b/c some rows 
    # don't have filter info (!) ...
    if np.in1d(sdss_name, sdss_ps1['dbID'].data)[0] : 
        row = np.where(sdss_ps1['dbID'].data == sdss_name)[0][0]
        ps1_name= sdss_ps1[row]['ps1id']
        print('  ps1 :  ',ps1_name)
        fname = ps1_dir + ps1_name
        f = open(fname, 'r')
        lines = f.readlines()
        f.close()
        lines_strip = [l.strip() for l in lines]
        lines_split = [l.split() for l in lines_strip]

        mjd = [] ; mag = [] ; magerr = [] ; filtr = []

        for line in lines_split[1:]:
            if len(line) > 3 : 
                mjd.append(float(line[0]))
                mag.append(float(line[1]))
                magerr.append(float(line[2]))
                filtr.append(line[3])
        if len(mjd) > 0 :  # there may be light curves that have no good measurement
            ps1_lc_ = Table([np.asarray(mjd), np.asarray(mag),
                           np.asarray(magerr), np.asarray(filtr)],
                          names=['mjd', 'mag', 'magerr', 'filter'])
            ps1_lc_['survey'] = 'ps1'
            # stack that
            lc = vstack([lc,ps1_lc_])

    # Save the light curve , only the good obs ...
    lc.write(out_dir+str(sdss_name.astype(int))+'.txt', format='ascii', 
             overwrite='True')
    
    i += 1 


0 sdss dbID:  70
  ps1 :   000840.63+011419.1
1 sdss dbID:  98
  ps1 :   000421.84+005743.6
2 sdss dbID:  233
  ptf :   225921000000762
  crts :   000119.50+001038.0.dat
  ps1 :   000119.50+001038.0
3 sdss dbID:  1018
  ptf :   225921080003697
  crts :   000527.52-000556.2.dat
  ps1 :   000527.52-000556.2
4 sdss dbID:  1310
  ps1 :   000053.17-001732.9
5 sdss dbID:  1327
  ptf :   26972060002245
  crts :   000107.52+004113.1.dat
  ps1 :   000107.52+004113.1
6 sdss dbID:  1540
  ptf :   26972100003354
  crts :   001025.90+005447.6.dat
  ps1 :   001025.90+005447.6
7 sdss dbID:  1580
  ptf :   26972080001361
  crts :   000640.67+003838.2.dat
  ps1 :   000640.67+003838.2
8 sdss dbID:  2113
  ptf :   225932080006687
  crts :   001924.06-001635.0.dat
  ps1 :   001924.06-001635.0
9 sdss dbID:  2306
  ptf :   26981070002072
  crts :   001606.24+002841.2.dat
  ps1 :   001606.24+002841.2
10 sdss dbID:  2401
  ptf :   225922090006176
  crts :   000654.10-001533.4.dat
  ps1 :   000654.10-001533.4


  crts :   001020.28-000738.8.dat
  ps1 :   001020.28-000738.8
85 sdss dbID:  25458
  ptf :   26971010000363
  crts :   000339.11+011019.6.dat
  ps1 :   000339.11+011019.6
86 sdss dbID:  25512
  ps1 :   000125.14+000009.4
87 sdss dbID:  25552
  ptf :   26972070000484
  crts :   000226.10+000549.1.dat
  ps1 :   000226.10+000549.1
88 sdss dbID:  25819
  crts :   001252.64-000206.0.dat
  ps1 :   001252.64-000206.0
89 sdss dbID:  26043
  ptf :   26981060000844
  crts :   001548.25+000827.1.dat
  ps1 :   001548.25+000827.1
90 sdss dbID:  26332
  ptf :   225932000004779
  crts :   001428.73+003904.6.dat
  ps1 :   001428.73+003904.6
91 sdss dbID:  26531
  ptf :   225922100006474
  crts :   000917.68-001344.8.dat
  ps1 :   000917.68-001344.8
92 sdss dbID:  26626
  ptf :   225922090002911
  crts :   000701.04-004435.0.dat
  ps1 :   000701.04-004435.0
93 sdss dbID:  27945
  ptf :   25922025001090
  crts :   000549.05-003435.5.dat
  ps1 :   000549.05-003435.5
94 sdss dbID:  28295
  ptf :   269711

  crts :   000451.78+011239.9.dat
  ps1 :   000451.78+011239.8
170 sdss dbID:  53132
  ptf :   26982070003409
  crts :   001557.08+004822.4.dat
  ps1 :   001557.08+004822.4
171 sdss dbID:  54008
  ptf :   225922090005585
  crts :   000822.06-002302.7.dat
  ps1 :   000822.07-002302.8
172 sdss dbID:  54088
  ptf :   26982010000301
  crts :   001647.83+011220.2.dat
  ps1 :   001647.83+011220.2
173 sdss dbID:  54554
  ptf :   225922090007038
  crts :   000750.05-000914.7.dat
  ps1 :   000750.05-000914.7
174 sdss dbID:  54929
  ptf :   25932010002591
  crts :   001747.79-001815.9.dat
  ps1 :   001747.79-001815.9
175 sdss dbID:  55571
  ptf :   26972100001749
  crts :   001027.42+002932.7.dat
  ps1 :   001027.42+002932.7
176 sdss dbID:  56217
  ptf :   225932060001896
  crts :   001436.17-005256.0.dat
  ps1 :   001436.17-005256.0
177 sdss dbID:  56423
  ptf :   225922080001193
  crts :   000553.71-010117.7.dat
  ps1 :   000553.71-010117.7
178 sdss dbID:  56486
  ptf :   25931010003953
  crts

  ps1 :   001816.53+004907.4
257 sdss dbID:  79609
  ptf :   26981010000616
  crts :   001632.10+011355.7.dat
  ps1 :   001632.10+011355.7
258 sdss dbID:  79963
  ptf :   26972090001289
  crts :   000807.53+001619.0.dat
  ps1 :   000807.53+001619.0
259 sdss dbID:  80524
  ptf :   225931070001926
  crts :   001616.72-004238.5.dat
  ps1 :   001616.72-004238.5
260 sdss dbID:  80641
  ptf :   225922040000614
  crts :   000912.89+000504.4.dat
  ps1 :   000912.89+000504.4
261 sdss dbID:  81005
  ps1 :   000541.88-010411.0
262 sdss dbID:  81061
  ptf :   26982070002172
  crts :   001559.58+004212.9.dat
  ps1 :   001559.58+004212.9
263 sdss dbID:  81140
  ptf :   225922080004984
  crts :   000500.42-003348.2.dat
  ps1 :   000500.42-003348.2
264 sdss dbID:  82001
  ps1 :   000046.15-003007.1
265 sdss dbID:  82447
  ps1 :   000330.19+000813.2
266 sdss dbID:  82552
  ptf :   25921080003741
  crts :   000435.75-011318.6.dat
  ps1 :   000435.75-011318.6
267 sdss dbID:  82616
  ps1 :   000608.04-010

  crts :   001502.26+001212.4.dat
  ps1 :   001502.26+001212.4
344 sdss dbID:  106772
  ptf :   225922010007731
  crts :   000248.92+005648.2.dat
  ps1 :   000248.92+005648.2
345 sdss dbID:  106913
  ptf :   225932020001139
  crts :   001921.73+000730.2.dat
  ps1 :   001921.73+000730.1
346 sdss dbID:  107313
  crts :   000912.07+005818.2.dat
  ps1 :   000912.08+005818.2
347 sdss dbID:  107687
  ps1 :   000348.46-004355.3
348 sdss dbID:  107867
  ptf :   225931070003132
  crts :   001805.51-002647.3.dat
  ps1 :   001805.51-002647.3
349 sdss dbID:  108155
  ptf :   225932070005357
  crts :   001711.56-002445.6.dat
  ps1 :   001711.56-002445.6
350 sdss dbID:  108734
  ptf :   225922100002167
  crts :   001004.96-004906.8.dat
  ps1 :   001004.96-004906.8
351 sdss dbID:  109176
  ps1 :   000657.63-010358.8
352 sdss dbID:  109511
  ptf :   225922080004773
  crts :   000453.25-003531.0.dat
  ps1 :   000453.26-003531.0
353 sdss dbID:  109616
  ps1 :   000042.02-004501.3
354 sdss dbID:  109828


...

  ps1 :   034121.58-010423.0
8753 sdss dbID:  4895864
  ps1 :   034137.03-000915.5
8754 sdss dbID:  4895984
  crts :   034009.46+001644.0.dat
  ps1 :   034009.47+001644.0
8755 sdss dbID:  4896021
  crts :   034115.84+004141.2.dat
  ps1 :   034115.84+004141.2
8756 sdss dbID:  4896450
  crts :   034958.20-005033.8.dat
  ps1 :   034958.20-005033.8
8757 sdss dbID:  4898421
  crts :   034419.83+001250.7.dat
  ps1 :   034419.83+001250.6
8758 sdss dbID:  4898550
  crts :   034102.34+010615.2.dat
  ps1 :   034102.34+010615.2
8759 sdss dbID:  4900845
  crts :   034210.15+004549.2.dat
  ps1 :   034210.15+004549.2
8760 sdss dbID:  4902031
  crts :   034428.69-003723.8.dat
  ps1 :   034428.69-003723.8
8761 sdss dbID:  4902215
  crts :   034306.57+000450.6.dat
  ps1 :   034306.57+000450.6
8762 sdss dbID:  4902270
  crts :   034317.60+005327.1.dat
  ps1 :   034317.60+005327.1
8763 sdss dbID:  4902282
  ps1 :   034629.06-000255.9
8764 sdss dbID:  4902719
  crts :   034116.54-005653.8.dat
  ps1 :   03

  ps1 :   034347.00-000706.5
8853 sdss dbID:  4954000
  ptf :   26092000001779
  crts :   035321.41-003333.9.dat
  ps1 :   035321.41-003333.9
8854 sdss dbID:  4954709
  crts :   034211.20-001331.9.dat
  ps1 :   034211.20-001331.9
8855 sdss dbID:  4955508
  crts :   034014.25+001847.7.dat
  ps1 :   034014.25+001847.7
8856 sdss dbID:  4955607
  ptf :   26082010001805
  crts :   034226.50-000427.1.dat
  ps1 :   034226.50-000427.1
8857 sdss dbID:  4956254
  crts :   034317.51-005920.6.dat
  ps1 :   034317.51-005920.6
8858 sdss dbID:  4956370
  crts :   034149.35-004917.9.dat
  ps1 :   034149.35-004917.9
8859 sdss dbID:  4957603
  crts :   034731.55-000027.2.dat
  ps1 :   034731.55-000027.2
8860 sdss dbID:  4959336
  ps1 :   034621.53+003325.3
8861 sdss dbID:  4960616
  ps1 :   034254.99-000621.2
8862 sdss dbID:  4960746
  crts :   034001.82+004118.6.dat
  ps1 :   034001.82+004118.6
8863 sdss dbID:  4961014
  crts :   034430.16+000407.3.dat
  ps1 :   034430.16+000407.3
8864 sdss dbID:  4961

  ps1 :   231000.16+003208.0
8951 sdss dbID:  7907462
  ptf :   27102060004636
  crts :   025959.68+004813.6.dat
  ps1 :   025959.68+004813.6
8952 sdss dbID:  7908238
  ptf :   1310012080000165
  crts :   221000.16-010925.3.dat
  ps1 :   221000.16-010925.3
8953 sdss dbID:  7910366
  ps1 :   024959.77-000104.2
8954 sdss dbID:  7911306
  ptf :   226012080001159
  crts :   021000.06-010208.8.dat
  ps1 :   021000.06-010208.8
8955 sdss dbID:  7911499
  ptf :   26882010000149
  crts :   215959.97-010747.7.dat
  ps1 :   215959.97-010747.7
8956 sdss dbID:  7913279
  ptf :   225942010005788
  crts :   003000.01+004400.8.dat
  ps1 :   003000.01+004400.8
8957 sdss dbID:  219418
  ps1 :   004642.31-000753.6
8958 sdss dbID:  223278
  ps1 :   004647.93-000617.4
8959 sdss dbID:  991367
  ps1 :   013138.18-001015.0
8960 sdss dbID:  884800
  ps1 :   013311.34-000117.3
8961 sdss dbID:  986181
  ps1 :   013703.27-000137.7
8962 sdss dbID:  1679638
  ps1 :   021415.26-000414.7
8963 sdss dbID:  1580050
  ps

9101 sdss dbID:  980072
  ps1 :   013523.49+001046.3
9102 sdss dbID:  892175
  ps1 :   013656.53+000041.6
9103 sdss dbID:  1662379
  ps1 :   021753.98+001126.3
9104 sdss dbID:  1703684
  ps1 :   021925.70+000302.5
9105 sdss dbID:  2044280
  ps1 :   022235.68+000252.6
9106 sdss dbID:  2041640
  ps1 :   022404.41+000506.5
9107 sdss dbID:  2003983
  ps1 :   022416.49+000121.2
9108 sdss dbID:  2042870
  ps1 :   022530.69+000914.1
9109 sdss dbID:  2031395
  ps1 :   022533.10+000308.3
9110 sdss dbID:  2089473
  ps1 :   022648.11+000724.6
9111 sdss dbID:  2093595
  ps1 :   022657.17+001157.2
9112 sdss dbID:  2028346
  ps1 :   022736.90+000716.7
9113 sdss dbID:  2119972
  ps1 :   022759.78+000947.4
9114 sdss dbID:  2027080
  ps1 :   022836.08+000939.2
9115 sdss dbID:  2499042
  ps1 :   025932.34+000953.2
9116 sdss dbID:  3131302
  ps1 :   030151.92+000558.4
9117 sdss dbID:  3784459
  ps1 :   033658.50+000347.3
9118 sdss dbID:  3757550
  ps1 :   033750.64+000012.1
9119 sdss dbID:  3722575
  ps1

  ps1 :   034127.35+011514.2
9257 sdss dbID:  350195
  ps1 :   233136.22+011206.6


In [52]:
sdss_name

70